In [ ]:
from datasets import load_from_disk, Dataset, DatasetDict
from typing import Union
train_local_dir:str = 'tiny_imagenet_local/train'
val_local_dir:str = 'tiny_imagenet_local/valid'
train_ds: Union[Dataset, DatasetDict] = load_from_disk(train_local_dir)
val_ds: Union[Dataset, DatasetDict] = load_from_disk(val_local_dir)
print(train_ds)
print(val_ds)

Dataset({
    features: ['image', 'label'],
    num_rows: 100000
})
Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})


In [24]:
for data in train_ds:
    print(data)
    break

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=64x64 at 0x2F9533511C0>, 'label': 0}


In [25]:
from PIL import Image
from typing import *
first_data:Dict = train_ds[0]
image:Image = first_data['image']
label:int = first_data['label']
#Check the image!
image.show()

In [26]:
#Using numpy since it cant be converted to torch tensors directly
import numpy as np
image_array:np.array = np.array(image)
print(f'Image array shape: {image_array.shape}')
print(f'Label: {label}')

Image array shape: (64, 64, 3)
Label: 0


In [30]:
import numpy as np
from PIL import Image

def mapper_fn(row):
    if isinstance(row['image'], Image.Image):
        # Convert PIL image to NumPy array, then to list
        row['image'] = np.array(row['image']).tolist()
    elif isinstance(row['image'], np.ndarray):
        # If already a NumPy array, convert to list
        row['image'] = row['image'].tolist()
    elif isinstance(row['image'], list):
        # If already a list, leave it as is
        pass
    elif row['image'] is None:
        # Handle None values (e.g., replace with empty list or placeholder)
        row['image'] = []
    else:
        # Catch unexpected types and force to list
        raise ValueError(f"Unexpected type for 'image': {type(row['image'])}")
    return row

# Apply the mapping
train_ds = train_ds.map(mapper_fn)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

ArrowInvalid: cannot mix list and non-list, non-null values

In [18]:
val_ds.map(mapper_fn)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})

In [21]:
for data in train_ds:
    print(data)
    break

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=64x64 at 0x2F94E17A600>, 'label': 0}


In [14]:
from torch.utils.data import DataLoader
train_dataloader: DataLoader = DataLoader(train_ds, batch_size = 8, shuffle = True)
val_dataloader: DataLoader = DataLoader(val_ds, batch_size = 8, shuffle = True)

In [16]:
for index, batch in enumerate(train_dataloader):
    print(f'Batch no {index+ 1}')
    print(batch)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.JpegImagePlugin.JpegImageFile'>